# Regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [26]:
from scipy import stats

In [2]:
df = pd.read_csv('Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

In [3]:
df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [82]:
male_df = df[~(df['segment'] == 'Womens E-Mail')]

In [83]:
male_df['treatment'] = male_df['segment'].apply(lambda x: 1 if x == 'Mens E-Mail' else 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [115]:
male_df.groupby('treatment').agg(conversion_rate=('conversion', 'mean'), spend_mean=('spend', 'mean'), count=('treatment', 'count'))

,conversion_rate,spend_mean,count
treatment,,,
0,0.005726,0.652789,21306
1,0.012531,1.422617,21307


In [116]:
# treatment == 1 の購買データ　と treatment == ０ の購買データのt test
stats.ttest_ind(male_df[male_df['treatment'] == 1]['spend'], male_df[male_df['treatment'] == 0]['spend'])

Ttest_indResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07)

### バイアスありのデータで有意差検定

In [135]:
treatment_df_tmp = male_df[male_df['treatment'] == 1]
control_df_tmp = male_df[male_df['treatment'] == 0]

treatment_df = treatment_df_tmp.drop(treatment_df_tmp[~(
    (treatment_df_tmp['history'] > 300) |
    (treatment_df_tmp['recency'] < 6) |
    (treatment_df_tmp['channel'] == 'Multichannel')
)].sample(frac=0.5, random_state=1).index)

control_df = control_df_tmp.drop(control_df_tmp[
    (control_df_tmp['history'] > 300) |
    (control_df_tmp['recency'] < 6) |
    (control_df_tmp['channel'] == 'Multichannel')
].sample(frac=0.5, random_state=1).index)

biased_df = pd.concat([treatment_df, control_df])
biased_df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1
16,5,1) $0 - $100,29.99,1,0,Surburban,0,Phone,Mens E-Mail,0,0,0.0,1
17,9,2) $100 - $200,112.35,1,0,Rural,0,Web,Mens E-Mail,0,0,0.0,1


In [136]:
biased_df.groupby('treatment').agg(conversion_rate = ('conversion','mean'), spend_mean = ('spend','mean'), count = ('treatment','count'))

,conversion_rate,spend_mean,count
treatment,,,
0,0.005760,0.697579,14756
1,0.012989,1.501725,17169


In [137]:
# treatment == 1 の購買データ　と treatment == ０ の購買データのt test
stats.ttest_ind(biased_df[biased_df['treatment'] == 1]['spend'], biased_df[male_df['treatment'] == 0]['spend'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Ttest_indResult(statistic=4.560664008782925, pvalue=5.118147589954738e-06)